In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import gc
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
df = pd.read_parquet('/kaggle/input/visual-taxonomy/category_attributes.parquet')
print(df.head())

In [ ]:
train_path = "/kaggle/input/visual-taxonomy/train.csv"
train_df = pd.read_csv(train_path)
train_df[train_df['Category'] == 'Women Tops & Tunics']

In [ ]:
l = df['Attribute_list'][4] #Women Tops and Tunics
for item in l :
    print(item)

In [ ]:
for i in range(1,11) :
    temp = train_df.groupby(f'attr_{i}').apply(len)
    print(temp) 
    print('--------------------------------------------')
    print('\n')
    
#color, fs, length, pattern, pp

In [ ]:
df_women = train_df.loc[train_df['Category'] == 'Women Tops & Tunics']
df_women.reset_index(drop=True, inplace=True)
df_women


In [ ]:
from keras.utils import normalize, to_categorical
pd.options.mode.chained_assignment = None
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from tqdm import tqdm
from keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [ ]:
df_women.isnull().sum()

In [ ]:
# pd.notna(3)

In [ ]:
import pandas as pd
import json

# Initialize a dictionary to store mappings for each attribute
mappings = {}

# Iterate through each attribute column to create and save mappings
for attr in df_women.columns[3:]:  # Adjust column range if necessary
    # Convert column to categorical and get the categories
    c = df_women[attr].astype('category')
    df_women[attr] = c.cat.codes  # Apply cat.codes to the DataFrame
    
    # Store the mapping of integer codes to original categories
    mappings[attr] = dict(enumerate(c.cat.categories))

# Display the mappings for verification
print("Mappings:")
for key, value in mappings.items():
    print(f"{key}: {value}")

# Save mappings to /kaggle/working/ for download
with open("/kaggle/working/attribute_mappings.json", "w") as f:
    json.dump(mappings, f, indent=4)

print("\nMappings saved to /kaggle/working/attribute_mappings.json")


In [ ]:
df_women['attr_1'].unique()

In [ ]:
# c = df_women['attr_4'].astype('category')

# d = dict(enumerate(c.cat.categories))
# print (d)
# {0: 'english', 1: 'spanish'}


In [ ]:
# df_women['attr_1'] = df_women["attr_1"].astype('category').cat.codes
# df_women['attr_2'] = df_women["attr_2"].astype('category').cat.codes
# df_women['attr_3'] = df_women["attr_3"].astype('category').cat.codes
# df_women['attr_4'] = df_women["attr_4"].astype('category').cat.codes
# df_women['attr_5'] = df_women["attr_5"].astype('category').cat.codes
# df_women['attr_6'] = df_women["attr_6"].astype('category').cat.codes
# df_women['attr_7'] = df_women["attr_7"].astype('category').cat.codes
# df_women['attr_8'] = df_women["attr_8"].astype('category').cat.codes
# df_women['attr_9'] = df_women["attr_9"].astype('category').cat.codes
# df_women['attr_10'] = df_women["attr_10"].astype('category').cat.codes

# color
# fit_shape
# length
# neck_collar
# ocassion
# pattern
# print_or_pattern_type
# sleeve_length
# sleeve_styling
# surface_styling


y_color = to_categorical(np.array(df_women['attr_1']))
y_fit_shape = to_categorical(np.array(df_women['attr_2']))
y_length = to_categorical(np.array(df_women['attr_3']))
y_neck_collar = to_categorical(np.array(df_women['attr_4']))
y_occasion = to_categorical(np.array(df_women['attr_5']))
y_pattern = to_categorical(np.array(df_women['attr_6']))
y_print_or_pattern_type = to_categorical(np.array(df_women['attr_7']))
y_sleeve_length = to_categorical(np.array(df_women['attr_8']))
y_sleeve_styling = to_categorical(np.array(df_women['attr_9']))
y_surface_styling = to_categorical(np.array(df_women['attr_10']))



In [ ]:
df_women

In [ ]:
print(y_color.shape)

In [ ]:
df_women['attr_10'].unique()

In [ ]:
image_directory = "/kaggle/input/visual-taxonomy/train_images/"
SIZE=200
X_dataset = []  
for i in tqdm(range(df_women.shape[0])):
    prep = str(df_women['id'][i])
    prep = '0'*(6-len(prep)) + prep
    img = image.load_img(image_directory +prep+'.jpg', target_size=(SIZE,SIZE,3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset.append(img)
X = np.array(X_dataset)



In [ ]:
X.shape

In [ ]:
len(df_women['attr_1'].unique())

In [ ]:
# color
# fit_shape
# length
# neck_collar
# ocassion
# pattern
# print_or_pattern_type
# sleeve_length
# sleeve_styling
# surface_styling



NUM_CLASSES_COLOR = len(df_women['attr_1'].unique())-1
NUM_CLASSES_FIT_SHAPE = len(df_women['attr_2'].unique())-1
NUM_CLASSES_LENGTH = len(df_women['attr_3'].unique())-1
NUM_CLASSES_NECK_COLLAR = len(df_women['attr_4'].unique())-1
NUM_CLASSES_OCCASION = len(df_women['attr_5'].unique())-1
NUM_CLASSES_PATTERN = len(df_women['attr_6'].unique())-1
NUM_CLASSES_PRINT_OR_PATTERN_TYPE = len(df_women['attr_7'].unique())-1
NUM_CLASSES_SLEEVE_LENGTH = len(df_women['attr_8'].unique())-1
NUM_CLASSES_SLEEVE_STYLING = len(df_women['attr_9'].unique())-1
NUM_CLASSES_SURFACE_STYLING = len(df_women['attr_10'].unique())-1


In [ ]:
IMAGE_SIZE = (200, 200, 3)


# Load the base model with pre-trained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMAGE_SIZE)

# Freeze the base model
base_model.trainable = True


In [ ]:
import keras 

In [ ]:
# Combine the categorical arrays into y_combined
y_combined = np.hstack((y_color, y_fit_shape, y_length, y_neck_collar, 
                        y_occasion, y_pattern, y_print_or_pattern_type, 
                        y_sleeve_length, y_sleeve_styling, y_surface_styling))

# Split into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y_combined,
    test_size=0.2,
    random_state=42
)

In [ ]:
del X
del train_df 
gc.collect()

In [ ]:
# Split the combined labels back into their respective arrays for the train set
train_color_labels = y_train[:, :NUM_CLASSES_COLOR]
train_fit_shape_labels = y_train[:, NUM_CLASSES_COLOR:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE]
train_length_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH]
train_neck_collar_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR]
train_occasion_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION]
train_pattern_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN]
train_print_or_pattern_type_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT_OR_PATTERN_TYPE]
train_sleeve_length_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT_OR_PATTERN_TYPE:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT_OR_PATTERN_TYPE + NUM_CLASSES_SLEEVE_LENGTH]
train_sleeve_styling_labels = y_train[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT_OR_PATTERN_TYPE + NUM_CLASSES_SLEEVE_LENGTH:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT_OR_PATTERN_TYPE + NUM_CLASSES_SLEEVE_LENGTH + NUM_CLASSES_SLEEVE_STYLING]
train_surface_styling_labels = y_train[:, -NUM_CLASSES_SURFACE_STYLING:]

# Split the combined labels back into their respective arrays for the validation set
val_color_labels = y_val[:, :NUM_CLASSES_COLOR]
val_fit_shape_labels = y_val[:, NUM_CLASSES_COLOR:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE]
val_length_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH]
val_neck_collar_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR]
val_occasion_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION]
val_pattern_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN]
val_print_or_pattern_type_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT_OR_PATTERN_TYPE]
val_sleeve_length_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT_OR_PATTERN_TYPE:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT_OR_PATTERN_TYPE + NUM_CLASSES_SLEEVE_LENGTH]
val_sleeve_styling_labels = y_val[:, NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT_OR_PATTERN_TYPE + NUM_CLASSES_SLEEVE_LENGTH:NUM_CLASSES_COLOR + NUM_CLASSES_FIT_SHAPE + NUM_CLASSES_LENGTH + NUM_CLASSES_NECK_COLLAR + NUM_CLASSES_OCCASION + NUM_CLASSES_PATTERN + NUM_CLASSES_PRINT_OR_PATTERN_TYPE + NUM_CLASSES_SLEEVE_LENGTH + NUM_CLASSES_SLEEVE_STYLING]
val_surface_styling_labels = y_val[:, -NUM_CLASSES_SURFACE_STYLING:]


In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 145

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Model input layer
inputs = layers.Input(shape=IMAGE_SIZE)
x = base_model(inputs, training=True)
x = layers.GlobalAveragePooling2D()(x)

# Define output layers for each attribute
output_color = layers.Dense(NUM_CLASSES_COLOR, activation='softmax', name='color')(x)
output_fit_shape = layers.Dense(NUM_CLASSES_FIT_SHAPE, activation='softmax', name='fit_shape')(x)
output_length = layers.Dense(NUM_CLASSES_LENGTH, activation='softmax', name='length')(x)
output_neck_collar = layers.Dense(NUM_CLASSES_NECK_COLLAR, activation='softmax', name='neck_collar')(x)
output_occasion = layers.Dense(NUM_CLASSES_OCCASION, activation='softmax', name='occasion')(x)
output_pattern = layers.Dense(NUM_CLASSES_PATTERN, activation='softmax', name='pattern')(x)
output_print_or_pattern_type = layers.Dense(NUM_CLASSES_PRINT_OR_PATTERN_TYPE, activation='softmax', name='print_or_pattern_type')(x)
output_sleeve_length = layers.Dense(NUM_CLASSES_SLEEVE_LENGTH, activation='softmax', name='sleeve_length')(x)
output_sleeve_styling = layers.Dense(NUM_CLASSES_SLEEVE_STYLING, activation='softmax', name='sleeve_styling')(x)
output_surface_styling = layers.Dense(NUM_CLASSES_SURFACE_STYLING, activation='softmax', name='surface_styling')(x)

# Define the model with all outputs
model = models.Model(
    inputs=inputs,
    outputs=[
        output_color, output_fit_shape, output_length, output_neck_collar,
        output_occasion, output_pattern, output_print_or_pattern_type,
        output_sleeve_length, output_sleeve_styling, output_surface_styling
    ]
)

# Compile the model with a loss function for each output
model.compile(
    optimizer=Adam(learning_rate=2*1e-5),
    loss={
        'color': 'categorical_crossentropy',
        'fit_shape': 'categorical_crossentropy',
        'length': 'categorical_crossentropy',
        'neck_collar': 'categorical_crossentropy',
        'occasion': 'categorical_crossentropy',
        'pattern': 'categorical_crossentropy',
        'print_or_pattern_type': 'categorical_crossentropy',
        'sleeve_length': 'categorical_crossentropy',
        'sleeve_styling': 'categorical_crossentropy',
        'surface_styling': 'categorical_crossentropy'
    },
    metrics={
        'color': 'accuracy',
        'fit_shape': 'accuracy',
        'length': 'accuracy',
        'neck_collar': 'accuracy',
        'occasion': 'accuracy',
        'pattern': 'accuracy',
        'print_or_pattern_type': 'accuracy',
        'sleeve_length': 'accuracy',
        'sleeve_styling': 'accuracy',
        'surface_styling': 'accuracy'
    }
)

# Print the model summary to review architecture and layers
model.summary()


In [ ]:
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=45,
    width_shift_range=0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [ ]:
class MultiLabelDataGenerator(Sequence):
    def __init__(self, images, labels, batch_size, datagen):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.datagen = datagen
        self.indices = np.arange(len(images))

    def __len__(self):
        return int(np.ceil(len(self.images) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images = self.images[batch_indices]
        batch_labels = {key: label[batch_indices] for key, label in self.labels.items()}
        
        # Apply data augmentation
        batch_images = np.array([self.datagen.random_transform(img) for img in batch_images])

        return batch_images, batch_labels

In [ ]:
train_labels = {
    'color': train_color_labels,
    'fit_shape': train_fit_shape_labels,
    'length': train_length_labels,
    'neck_collar': train_neck_collar_labels,
    'occasion': train_occasion_labels,
    'pattern': train_pattern_labels,
    'print_or_pattern_type': train_print_or_pattern_type_labels,
    'sleeve_length': train_sleeve_length_labels,
    'sleeve_styling': train_sleeve_styling_labels,
    'surface_styling': train_surface_styling_labels
}

batch_size = 32
train_generator = MultiLabelDataGenerator(X_train, train_labels, batch_size, train_datagen)


In [ ]:
checkpoint_filepath = '/kaggle/working/checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_color_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = model.fit(
    train_generator,  # Image data
    validation_data=(X_val, {
        'color': val_color_labels,
        'fit_shape': val_fit_shape_labels,
        'length': val_length_labels,
        'neck_collar': val_neck_collar_labels,
        'occasion': val_occasion_labels,
        'pattern': val_pattern_labels,
        'print_or_pattern_type': val_print_or_pattern_type_labels,
        'sleeve_length': val_sleeve_length_labels,
        'sleeve_styling': val_sleeve_styling_labels,
        'surface_styling': val_surface_styling_labels
    }),
    epochs=50,
    callbacks=[model_checkpoint_callback]
)


In [ ]:
import matplotlib.pyplot as plt

# Assuming 'history' is the output from model.fit()
# Example: history = model.fit(...)

# Plotting the accuracy
plt.figure(figsize=(12, 6))

# Plot training accuracy for each attribute
plt.plot(history.history['color_accuracy'], label='Color Train Accuracy', color='red')
plt.plot(history.history['fit_shape_accuracy'], label='Fit Shape Train Accuracy', color='blue')
plt.plot(history.history['length_accuracy'], label='Length Train Accuracy', color='green')
plt.plot(history.history['neck_collar_accuracy'], label='Neck Collar Train Accuracy', color='orange')
plt.plot(history.history['occasion_accuracy'], label='Occasion Train Accuracy', color='purple')
plt.plot(history.history['pattern_accuracy'], label='Pattern Train Accuracy', color='brown')
plt.plot(history.history['print_or_pattern_type_accuracy'], label='Print or Pattern Type Train Accuracy', color='pink')
plt.plot(history.history['sleeve_length_accuracy'], label='Sleeve Length Train Accuracy', color='cyan')
plt.plot(history.history['sleeve_styling_accuracy'], label='Sleeve Styling Train Accuracy', color='magenta')
plt.plot(history.history['surface_styling_accuracy'], label='Surface Styling Train Accuracy', color='gray')

# Plot validation accuracy for each attribute
plt.plot(history.history['val_color_accuracy'], label='Color Validation Accuracy', linestyle='dashed', color='red')
plt.plot(history.history['val_fit_shape_accuracy'], label='Fit Shape Validation Accuracy', linestyle='dashed', color='blue')
plt.plot(history.history['val_length_accuracy'], label='Length Validation Accuracy', linestyle='dashed', color='green')
plt.plot(history.history['val_neck_collar_accuracy'], label='Neck Collar Validation Accuracy', linestyle='dashed', color='orange')
plt.plot(history.history['val_occasion_accuracy'], label='Occasion Validation Accuracy', linestyle='dashed', color='purple')
plt.plot(history.history['val_pattern_accuracy'], label='Pattern Validation Accuracy', linestyle='dashed', color='brown')
plt.plot(history.history['val_print_or_pattern_type_accuracy'], label='Print or Pattern Type Validation Accuracy', linestyle='dashed', color='pink')
plt.plot(history.history['val_sleeve_length_accuracy'], label='Sleeve Length Validation Accuracy', linestyle='dashed', color='cyan')
plt.plot(history.history['val_sleeve_styling_accuracy'], label='Sleeve Styling Validation Accuracy', linestyle='dashed', color='magenta')
plt.plot(history.history['val_surface_styling_accuracy'], label='Surface Styling Validation Accuracy', linestyle='dashed', color='gray')

# Adding labels and title
plt.title('Training and Validation Accuracy for Clothing Attributes')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
os.chdir(r'/kaggle/working')

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink(r'/kaggle/working/checkpoint.model.keras')

## Inferencing - In progress

In [ ]:
from tensorflow.keras.models import load_model

# Load the best model
best_model = load_model('/kaggle/working/checkpoint.model.keras')


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Assume df_women is already defined with your data structure
# Create category mappings from attr columns
blouse_pattern_classes = df_women['attr_1'].unique()
border_classes = df_women['attr_2'].unique()
border_width_classes = df_women['attr_3'].unique()
color_classes = df_women['attr_4'].unique()
occasion_classes = df_women['attr_5'].unique()
ornamentation_classes = df_women['attr_6'].unique()
pallu_details_classes = df_women['attr_7'].unique()
pattern_classes = df_women['attr_8'].unique()
print_or_pattern_type_classes = df_women['attr_9'].unique()
transparency_classes = df_women['attr_10'].unique()

In [ ]:
blouse_pattern_classes.sort()
blouplasses

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Select an index for a validation image
index = 1  # Change this to test different images

# Use the validation image from X_val
new_image = X_val[index:index + 1]  # Shape (1, IMAGE_HEIGHT, IMAGE_WIDTH, 3)

# Get the true labels for this validation image
true_blouse_pattern_label = val_blouse_pattern_labels[index]
true_border_label = val_border_labels[index]
true_border_width_label = val_border_width_labels[index]
true_color_label = val_color_labels[index]
true_occasion_label = val_occasion_labels[index]
true_ornamentation_label = val_ornamentation_labels[index]
true_pallu_details_label = val_pallu_details_labels[index]
true_pattern_label = val_pattern_labels[index]
true_print_or_pattern_type_label = val_print_or_pattern_type_labels[index]
true_transparency_label = val_transparency_labels[index]

# Predict attribute values
predictions = best_model.predict(new_image)
predicted_blouse_pattern = np.argmax(predictions[0])
predicted_border = np.argmax(predictions[1])
predicted_border_width = np.argmax(predictions[2])
predicted_color = np.argmax(predictions[3])
predicted_occasion = np.argmax(predictions[4])
predicted_ornamentation = np.argmax(predictions[5])
predicted_pallu_details = np.argmax(predictions[6])
predicted_pattern = np.argmax(predictions[7])
predicted_print_or_pattern_type = np.argmax(predictions[8])
predicted_transparency = np.argmax(predictions[9])

# Print predicted attributes
print("Predicted attributes:")
print("Blouse Pattern:", blouse_pattern_classes[predicted_blouse_pattern])
print("Border:", border_classes[predicted_border])
print("Border Width:", border_width_classes[predicted_border_width])
print("Color:", color_classes[predicted_color])
print("Occasion:", occasion_classes[predicted_occasion])
print("Ornamentation:", ornamentation_classes[predicted_ornamentation])
print("Pallu Details:", pallu_details_classes[predicted_pallu_details])
print("Pattern:", pattern_classes[predicted_pattern])
print("Print or Pattern Type:", print_or_pattern_type_classes[predicted_print_or_pattern_type])
print("Transparency:", transparency_classes[predicted_transparency])

# Print true labels
print("\nTrue attributes:")
print("Blouse Pattern:", blouse_pattern_classes[np.argmax(true_blouse_pattern_label)])
print("Border:", border_classes[np.argmax(true_border_label)])
print("Border Width:", border_width_classes[np.argmax(true_border_width_label)])
print("Color:", color_classes[np.argmax(true_color_label)])
print("Occasion:", occasion_classes[np.argmax(true_occasion_label)])
print("Ornamentation:", ornamentation_classes[np.argmax(true_ornamentation_label)])
print("Pallu Details:", pallu_details_classes[np.argmax(true_pallu_details_label)])
print("Pattern:", pattern_classes[np.argmax(true_pattern_label)])
print("Print or Pattern Type:", print_or_pattern_type_classes[np.argmax(true_print_or_pattern_type_label)])
print("Transparency:", transparency_classes[np.argmax(true_transparency_label)])

# Plot the validation image
plt.imshow(new_image[0])
plt.axis('off')
plt.show()
